In [1]:
# Check if GPU is available
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
# load the dataset for fine-tuning
from datasets import load_dataset

# Load DailyDialog dataset
dataset = load_dataset("daily_dialog", trust_remote_code=True)

Using the latest cached version of the module from C:\Users\chayk\.cache\huggingface\modules\datasets_modules\datasets\daily_dialog\1d0a58c7f2a4dab5ed9d01dbde8e55e0058e589ab81fce5c2df929ea810eabcd (last modified on Mon Jun  3 01:14:33 2024) since it couldn't be found locally at daily_dialog, or remotely on the Hugging Face Hub.


In [3]:
# load initial model
from transformers import AutoTokenizer

# Load the tokenizer for DialoGPT
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'}

In [4]:
tokenizer.add_special_tokens({'pad_token': '<|PAD|>'})
tokenizer.add_special_tokens({'unk_token': '<|UNK|>'})
tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|UNK|>',
 'pad_token': '<|PAD|>'}

In [5]:
def preprocess_function(examples):
    input_ids_list = []
    for dialog in examples['dialog']:
        full_conversation = ""
        for turn in dialog:
            full_conversation += turn + tokenizer.eos_token
        input_ids = tokenizer(full_conversation, return_tensors="pt", truncation=True, max_length=1024).input_ids
        input_ids_list.append(input_ids.squeeze())

    return {"input_ids": input_ids_list}

In [6]:
# Preprocess the dataset
train_dataset = dataset["train"].map(preprocess_function, batched=True, remove_columns=["dialog", "act", "emotion"])
valid_dataset = dataset["validation"].map(preprocess_function, batched=True, remove_columns=["dialog", "act", "emotion"])

Map:   0%|          | 0/11118 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, pad_to_multiple_of=8)

In [8]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments

# Load the pre-trained DialoGPT model
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium").to(device)
# Resize the token embeddings
model.resize_token_embeddings(len(tokenizer))

# Check the number of tokens
print(f"Tokenizer vocab size: {len(tokenizer)}")
print(f"Model's embedding layer size: {model.transformer.wte.num_embeddings}")

Tokenizer vocab size: 50259
Model's embedding layer size: 50259


In [9]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=500,
    fp16=True,
    dataloader_num_workers=2
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

# Fine-tune the model
trainer.train()

d:\Anaconda\envs\my_ml_env\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/37060 [00:00<?, ?it/s]

{'loss': 2.6752, 'grad_norm': 1.4994325637817383, 'learning_rate': 4.932811656772801e-05, 'epoch': 0.13}
{'loss': 2.4475, 'grad_norm': 1.636724591255188, 'learning_rate': 4.865353480841878e-05, 'epoch': 0.27}
{'loss': 2.3843, 'grad_norm': 1.6692636013031006, 'learning_rate': 4.7978953049109555e-05, 'epoch': 0.4}
{'loss': 2.3553, 'grad_norm': 1.7826321125030518, 'learning_rate': 4.7304371289800323e-05, 'epoch': 0.54}
{'loss': 2.3392, 'grad_norm': 1.679495930671692, 'learning_rate': 4.66297895304911e-05, 'epoch': 0.67}
{'loss': 2.2918, 'grad_norm': 2.038083076477051, 'learning_rate': 4.595520777118187e-05, 'epoch': 0.81}
{'loss': 2.2631, 'grad_norm': 2.4135501384735107, 'learning_rate': 4.528062601187264e-05, 'epoch': 0.94}


  0%|          | 0/334 [00:00<?, ?it/s]

{'eval_loss': 2.1729540824890137, 'eval_runtime': 16.5919, 'eval_samples_per_second': 60.27, 'eval_steps_per_second': 20.13, 'epoch': 1.0}
{'loss': 2.1108, 'grad_norm': 1.7540032863616943, 'learning_rate': 4.460604425256341e-05, 'epoch': 1.08}
{'loss': 1.9894, 'grad_norm': 1.4170247316360474, 'learning_rate': 4.393146249325418e-05, 'epoch': 1.21}
{'loss': 1.9442, 'grad_norm': 1.843526005744934, 'learning_rate': 4.3256880733944956e-05, 'epoch': 1.35}
{'loss': 1.9381, 'grad_norm': 2.150355100631714, 'learning_rate': 4.2582298974635725e-05, 'epoch': 1.48}
{'loss': 1.9525, 'grad_norm': 1.3762195110321045, 'learning_rate': 4.190906637884512e-05, 'epoch': 1.62}
{'loss': 1.9475, 'grad_norm': 2.7121331691741943, 'learning_rate': 4.123448461953589e-05, 'epoch': 1.75}
{'loss': 1.9528, 'grad_norm': 2.4378883838653564, 'learning_rate': 4.055990286022666e-05, 'epoch': 1.89}


...

  0%|          | 0/334 [00:00<?, ?it/s]

{'eval_loss': 2.405756950378418, 'eval_runtime': 16.5721, 'eval_samples_per_second': 60.342, 'eval_steps_per_second': 20.154, 'epoch': 9.0}
{'loss': 0.7721, 'grad_norm': 2.5153489112854004, 'learning_rate': 4.819212088505127e-06, 'epoch': 9.04}
{'loss': 0.7424, 'grad_norm': 3.14442777633667, 'learning_rate': 4.144630329195899e-06, 'epoch': 9.17}
{'loss': 0.7445, 'grad_norm': 2.7694032192230225, 'learning_rate': 3.4700485698866704e-06, 'epoch': 9.31}
{'loss': 0.7278, 'grad_norm': 2.410040855407715, 'learning_rate': 2.7968159740960605e-06, 'epoch': 9.44}
{'loss': 0.7234, 'grad_norm': 2.927440881729126, 'learning_rate': 2.122234214786832e-06, 'epoch': 9.58}
{'loss': 0.7547, 'grad_norm': 2.010244131088257, 'learning_rate': 1.447652455477604e-06, 'epoch': 9.71}
{'loss': 0.7477, 'grad_norm': 2.601353406906128, 'learning_rate': 7.730706961683757e-07, 'epoch': 9.85}
{'loss': 0.7347, 'grad_norm': 2.679176092147827, 'learning_rate': 1.0118726389638424e-07, 'epoch': 9.98}


  0%|          | 0/334 [00:00<?, ?it/s]

{'eval_loss': 2.4457435607910156, 'eval_runtime': 14.4988, 'eval_samples_per_second': 68.971, 'eval_steps_per_second': 23.036, 'epoch': 10.0}
{'train_runtime': 7181.676, 'train_samples_per_second': 15.481, 'train_steps_per_second': 5.16, 'train_loss': 1.3019859219266476, 'epoch': 10.0}


TrainOutput(global_step=37060, training_loss=1.3019859219266476, metrics={'train_runtime': 7181.676, 'train_samples_per_second': 15.481, 'train_steps_per_second': 5.16, 'total_flos': 4.163011146429235e+16, 'train_loss': 1.3019859219266476, 'epoch': 10.0})

In [10]:
# Save the model and tokenizer
model.save_pretrained("./models/fine-tuned-dialogpt_new")
tokenizer.save_pretrained("./models/fine-tuned-dialogpt_new")

('./fine-tuned-dialogpt_new\\tokenizer_config.json',
 './fine-tuned-dialogpt_new\\special_tokens_map.json',
 './fine-tuned-dialogpt_new\\vocab.json',
 './fine-tuned-dialogpt_new\\merges.txt',
 './fine-tuned-dialogpt_new\\added_tokens.json',
 './fine-tuned-dialogpt_new\\tokenizer.json')